In [ ]:
from googleAPI import *
from pymongo import MongoClient
from crawling import ProgressBar

In [ ]:
client = MongoClient(
    "mongodb+srv://cluster0.c6ccx.mongodb.net",
    authsource="$external",
    authmechanism="MONGODB-X509",
    tls=True,
    tlsCertificateKeyFile="../../../mongodb.pem",
)
db = client["youtube"]
searchData = db["searchData"]

In [ ]:
with open("apikey.txt", encoding="utf8") as f:
    apikey1 = f.readline().strip()
    apikey2 = f.readline().strip()

In [ ]:
import datetime

global_query = {"key": apikey1, "maxResults": 100, "regionCode": "KR"}

# 1970-01-01T00:00:00Z
def strftime(datetime: datetime.datetime):
    return datetime.strftime("%Y-%m-%dT%H:%M:%SZ")


def strptime(str):
    return datetime.datetime().strptime(str, "%Y-%m-%dT%H:%M:%SZ")


# search with keyword(costs to much(100 times much than other))
def get_searched_video(query):
    final_query = {**global_query, **query, "part": "id, snippet", "type": "video", "safeSearch": "strict"}
    return get_youtube_api_json("search", final_query)


def get_populer_video(query):
    return get_video({**query, "chart": "mostPopular"})


def get_video(query):
    final_query = {
        **global_query,
        **query,
        "part": "snippet, contentDetails, status, statistics, player, topicDetails, recordingDetails",
    }
    return get_youtube_api_json("videos", final_query)


def get_id_from_result(result):
    items = result.get("items", [])
    ids = []
    for item in items:
        if item.get("id"):
            if type(item) == str:
                ids.append(item["id"])
            elif type(item) == dict and item["id"].get("videoId"):
                ids.append(item["id"]["videoId"])
    return ids


def flatten_function(api_function, query):
    nextPageToken = ""
    total_result = []
    while nextPageToken is not None:
        print("check1")
        query["pageToken"] = nextPageToken
        result = api_function(query)
        if type(result.get("items")) == list:
            print("check2")
            total_result.extend(result["items"])
        nextPageToken = result.get("nextPageToken")    
    print("total_result####################################################", total_result)
    return {**query, "result": total_result}


def flatten_data(results, query):
    if type(results) == dict:
        results = [results]
    total_result = []
    for result in results:
        if type(result.get("items")) == list:
            total_result.extend(result["items"])
    return {**query, "result": total_result}


def get_rebuild_searched_video(query):
    init_data = flatten_function(get_searched_video, query)["result"]
    print("init_data##########################################################", init_data)
    data_size = len(init_data)
    total_data = []
    temp = 0
    while data_size > temp * 50:
        ids = [data["id"]["videoId"] for data in init_data[temp * 50 : min((temp + 1) * 50, data_size)]]
        total_data.append(get_video({"id": ",".join(ids)}))
        temp += 1
    print("total_data#########################################################", total_data)
    return flatten_data(total_data, query)


In [ ]:
get_video({"id":"XqZsoesa55w"})

In [ ]:
start_date = strftime(datetime.datetime(2022, 2, 1))
end_date = strftime(datetime.datetime(2022, 3, 1))
mydata = get_rebuild_searched_video({"publishedAfter": start_date, "publishedBefore": end_date})
print(mydata)